In [140]:
import numpy as np
import pandas as pd

from random import random
from scipy.optimize import linprog

# Прямая задача

In [141]:
def simplex(A, b, c):
    n, m = A.shape

    table = np.zeros((n, m + n), dtype=float)

    for i in range(m):
        for j in range(n):
            table[j, i] = A[j, i]

    for i in range(m, n + m):
        for j in range(n):
            if i - m == j:
                table[j, i] = 1
            else:
                table[j, i] = 0

    b = b

    c_old = c.copy()

    c = np.zeros(n + m, dtype=float)
    for i in range(m):
        c[i] = c_old[i]

    Cb = np.zeros(n, dtype=float)
    Bp = np.array([i for i in range(m, n + m)], dtype=float)  

    result = np.zeros(n + m)

    while True:
        F = sum(Cb * b)

        simplex_matrix = np.zeros((n, 4 + m + n), dtype=float)
        simplex_matrix[:, 0:1] = Bp.reshape(n, 1)
        simplex_matrix[:, 1:2] = Cb.reshape(n, 1)
        simplex_matrix[:, 2:3] = b.reshape(n, 1)
        simplex_matrix[:, 3:-1] = table

        deltas = np.array([c[i] - sum(Cb[j] * table[j, i] for j in range(n)) for i in range(n + m)], dtype=float)
        
        print(f'Промежуточный F_max = {np.round(F, 2)}')

        k = np.argmax(deltas)

        fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)
        fractions = np.array([fractions[i] if fractions[i] > 0 else np.inf for i in range(n)], dtype=float)

        simplex_matrix[:, n + m + 3:n + m + 4] = fractions.reshape(n, 1)

        print(f'Δ = {np.round(deltas, 2)}')

        flag = True
        for i in range(n + m):
            if deltas[i] > 0:
                flag = False
                break
        
        if flag:
            return np.round(F, 2), np.round(result, 2)

        print(f'simpex table: \n{pd.DataFrame(np.round(simplex_matrix, 2))}')

        r = np.argmin(fractions)

        result[k] = fractions[r] * deltas[k] / c[k]

        table_old = table.copy()

        for i in range(n + m):
            for j in range(n):
                if j == r:
                    table[j, i] = table_old[j, i] / table_old[r, k]
                else:
                    table[j, i] = table_old[j, i] - (table_old[r, i] * table_old[j, k] / table_old[r, k])

        Bp[r] = k
        Cb[r] = c[k]

        b_old = b.copy()
        for i in range(n + m):
            for j in range(n):
                if j == r:
                    b[j] = b_old[j] / table_old[r, k]
                else:
                    b[j] = b_old[j] - (b_old[r] * table_old[j, k] / table_old[r, k])

        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')

## Тестовые данные

In [142]:
c = np.array([4, 5], dtype=float) 

A = np.array([
    [2, 4],
    [1, 1],
    [2, 1]
], dtype=float)

b = np.array([560, 170, 300], dtype=float)

F_max, result = simplex(A, b, c)
print(f'F_max = {F_max}\nОптимальные значения переменных: \n{result}')

Промежуточный F_max = 0.0
Δ = [4. 5. 0. 0. 0.]
simpex table: 
     0    1      2    3    4    5    6    7      8
0  2.0  0.0  560.0  2.0  4.0  1.0  0.0  0.0  140.0
1  3.0  0.0  170.0  1.0  1.0  0.0  1.0  0.0  170.0
2  4.0  0.0  300.0  2.0  1.0  0.0  0.0  1.0  300.0
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Промежуточный F_max = 700.0
Δ = [ 1.5   0.   -1.25  0.    0.  ]
simpex table: 
     0    1      2    3    4     5    6    7       8
0  1.0  5.0  140.0  0.5  1.0  0.25  0.0  0.0  280.00
1  3.0  0.0   30.0  0.5  0.0 -0.25  1.0  0.0   60.00
2  4.0  0.0  160.0  1.5  0.0 -0.25  0.0  1.0  106.67
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Промежуточный F_max = 790.0
Δ = [ 0.   0.  -0.5 -3.   0. ]
F_max = 790.0
Оптимальные значения переменных: 
[ 22.5 140.    0.    0.    0. ]


C:\Users\User\AppData\Local\Temp\ipykernel_7888\2482700145.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)


## Случайные данные

In [143]:
c = np.array([2 + round(2 * random(), 1) for x in range(6)], dtype=float)  
print(F'{c=}\n')

A = np.array([[2 + round(15 * random(), 1) for x in range(6)] for y in range(8)], dtype=float)
print(F'A={pd.DataFrame(A)}\n')

b = np.array([50 + round(50 * random(), 0) for y in range(8)], dtype=float)
print(F'{b=}\n')

c=array([3. , 2.7, 3. , 3.8, 2. , 3.5])

A=      0     1     2     3     4     5
0   4.2  12.4  11.0   4.8   7.1   8.5
1   7.8  16.8  16.8   4.1   8.7  13.6
2   7.2  14.6  15.0  13.8   5.7   9.4
3  15.4  14.2   3.3  16.6   9.8   5.1
4  16.1   6.6   8.8   7.1   8.9   2.3
5  12.3   3.0   4.0  12.2   7.5  16.0
6   7.1   8.9   5.1   7.5   2.6   5.8
7   6.5  15.3  11.3   8.0  16.8  15.3

b=array([67., 61., 59., 68., 59., 63., 88., 89.])



In [144]:
F_max, result = simplex(A, b, c)
print(f'F_max = {F_max}\nОптимальные значения переменных: \n{result}')
sum(result[:c.shape[0]] * c)

Промежуточный F_max = 0.0
Δ = [3.  2.7 3.  3.8 2.  3.5 0.  0.  0.  0.  0.  0.  0.  0. ]
simpex table: 
     0    1     2     3     4     5     6     7     8    9    10   11   12  \
0   6.0  0.0  67.0   4.2  12.4  11.0   4.8   7.1   8.5  1.0  0.0  0.0  0.0   
1   7.0  0.0  61.0   7.8  16.8  16.8   4.1   8.7  13.6  0.0  1.0  0.0  0.0   
2   8.0  0.0  59.0   7.2  14.6  15.0  13.8   5.7   9.4  0.0  0.0  1.0  0.0   
3   9.0  0.0  68.0  15.4  14.2   3.3  16.6   9.8   5.1  0.0  0.0  0.0  1.0   
4  10.0  0.0  59.0  16.1   6.6   8.8   7.1   8.9   2.3  0.0  0.0  0.0  0.0   
5  11.0  0.0  63.0  12.3   3.0   4.0  12.2   7.5  16.0  0.0  0.0  0.0  0.0   
6  12.0  0.0  88.0   7.1   8.9   5.1   7.5   2.6   5.8  0.0  0.0  0.0  0.0   
7  13.0  0.0  89.0   6.5  15.3  11.3   8.0  16.8  15.3  0.0  0.0  0.0  0.0   

    13   14   15   16     17  
0  0.0  0.0  0.0  0.0  13.96  
1  0.0  0.0  0.0  0.0  14.88  
2  0.0  0.0  0.0  0.0   4.28  
3  0.0  0.0  0.0  0.0   4.10  
4  1.0  0.0  0.0  0.0   8.31  
5  0.0  

C:\Users\User\AppData\Local\Temp\ipykernel_7888\2482700145.py:65: RuntimeWarning: divide by zero encountered in scalar divide
  result[k] = fractions[r] * deltas[k] / c[k]
C:\Users\User\AppData\Local\Temp\ipykernel_7888\2482700145.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)


np.float64(18.29)

# Двойственная задача

In [145]:
def dual_simplex(A, b, c, M):
    
    def get_optimal_values(Bp, b, n, m):
        optimal_values = np.zeros(2 * n + m)  # Creates an array to hold the optimal values
        for i in range(len(Bp)):
            if Bp[i] < len(optimal_values):  # Ensure we do not go out of bounds
                optimal_values[int(Bp[i])] = b[i]
        return optimal_values

    n, m = A.shape

    table = np.zeros((n, m + 2 * n), dtype=float)

    for i in range(m):
        for j in range(n):
            table[j, i] = A[j, i]

    for i in range(m, n + m):
        for j in range(n):
            if i - m == j:
                table[j, i] = -1
            else:
                table[j, i] = 0

    for i in range(n + m, 2 * n + m):
        for j in range(n):
            if i - m - n == j:
                table[j, i] = 1
            else:
                table[j, i] = 0

    b = b

    c_old = c.copy()

    c = np.zeros(2 * n + m, dtype=float)
    for i in range(m):
        c[i] = c_old[i]
    for i in range(n + m, 2 * n + m):
        c[i] = M

    Cb = np.array([M for _ in range(n)], dtype=float)
    Bp = np.array([i for i in range(m + n, m + 2 * n)], dtype=float) 

    while True:
        F = sum(Cb * b)

        simplex_matrix = np.zeros((n, 4 + m + 2 * n), dtype=float)
        simplex_matrix[:, 0:1] = Bp.reshape(n, 1)
        simplex_matrix[:, 1:2] = Cb.reshape(n, 1)
        simplex_matrix[:, 2:3] = b.reshape(n, 1)
        simplex_matrix[:, 3:-1] = table

        deltas = np.array([c[i] - sum(Cb[j] * table[j, i] for j in range(n)) for i in range(n + m)], dtype=float)
        
        print(f'Промежуточный F_min = {np.round(F, 2)}')

        k = np.argmin(deltas)

        fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)
        fractions = np.array([fractions[i] if fractions[i] > 0 else np.inf for i in range(n)], dtype=float)

        simplex_matrix[:, 2 * n + m + 3: 2 * n + m + 4] = fractions.reshape(n, 1)

        print(f'Δ = {np.round(deltas, 2)}')

        flag = True
        for i in range(n + m):
            if deltas[i] < 0:
                flag = False
                break
        
        if flag:
            return np.round(F, 2), np.round(get_optimal_values(Bp, b, n, m), 2)
        
        print(f'simpex table: \n{pd.DataFrame(np.round(simplex_matrix, 2))}')

        r = np.argmin(fractions)

        table_old = table.copy()

        for i in range(2 * n + m):
            for j in range(n):
                if j == r:
                    table[j, i] = table_old[j, i] / table_old[r, k]
                else:
                    table[j, i] = table_old[j, i] - (table_old[r, i] * table_old[j, k] / table_old[r, k])

        Bp[r] = k
        Cb[r] = c[k]

        b_old = b.copy()
        for i in range(n + m):
            for j in range(n):
                if j == r:
                    b[j] = b_old[j] / table_old[r, k]
                else:
                    b[j] = b_old[j] - (b_old[r] * table_old[j, k] / table_old[r, k])

        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')

## Тестовые данные

In [146]:
M = 10 ** 3

c = np.array([560, 170, 300], dtype=float)

A = np.array([
    [2, 1, 2],
    [4, 1, 1]
], dtype=float)

b = np.array([4, 5], dtype=float)

In [147]:
F_min, result = dual_simplex(A, b, c, M=M)
print(f'F_min = {F_min}')
print(f'Оптимальное значение переменных:\n{result}')

Промежуточный F_min = 9000.0
Δ = [-5440. -1830. -2700.  1000.  1000.]
simpex table: 
    0       1    2    3    4    5    6    7    8    9     10
0  5.0  1000.0  4.0  2.0  1.0  2.0 -1.0  0.0  1.0  0.0  2.00
1  6.0  1000.0  5.0  4.0  1.0  1.0  0.0 -1.0  0.0  1.0  1.25
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Промежуточный F_min = 2200.0
Δ = [    0.  -470. -1340.  1000.  -360.]
simpex table: 
    0       1     2    3     4     5    6     7    8     9    10
0  5.0  1000.0  1.50  0.0  0.50  1.50 -1.0  0.50  1.0 -0.50  1.0
1  0.0   560.0  1.25  1.0  0.25  0.25  0.0 -0.25  0.0  0.25  5.0
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
Промежуточный F_min = 860.0
Δ = [  0.   -23.33   0.   106.67  86.67]
simpex table: 
    0      1    2    3     4    5     6     7     8     9    10
0  2.0  300.0  1.0  0.0  0.33  1.0 -0.67  0.33  0.67 -0.33  3.0
1  0.0  560.0  1.0  1.0  0.17  0.0  0.17 -0.33 -0.17  0.33  6.0
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

C:\Users\User\AppData\Local\Temp\ipykernel_7888\666884200.py:60: RuntimeWarning: divide by zero encountered in scalar divide
  fractions = np.array([b[i] / table[i, k] for i in range(n)], dtype=float)


Среди $\Delta_i$ не осталось отрицательных $\implies$ решение оптимально

Оптимальные значение переменных искусственного базиса $=0\implies$ решение допустимо

## Случайные данные

In [148]:
c = np.array([2 + round(2 * random(), 1) for x in range(6)], dtype=float)  
print(F'{c=}\n')

A = np.array([[2 + round(15 * random(), 1) for x in range(6)] for y in range(8)], dtype=float)
print(F'A={pd.DataFrame(A)}\n')

b = np.array([50 + round(50 * random(), 0) for y in range(8)], dtype=float)
print(F'{b=}\n')

c=array([2.6, 3.1, 2.2, 3.7, 3.3, 4. ])

A=      0     1     2     3     4     5
0   4.3  13.7  12.9   6.4  13.3   8.9
1   7.3  12.6   9.9   7.4  16.8  15.8
2  15.0  14.1   4.0   8.3   7.9  15.6
3  11.6  11.3   8.6  15.5  11.4   7.3
4  15.3   3.3   9.6  10.0   4.6  11.9
5   7.7  12.9   3.7   4.3  13.9  11.2
6  14.6  16.4  15.0   9.2  12.9  16.3
7   9.5  12.5   4.1   7.3   9.5  10.8

b=array([92., 95., 64., 97., 98., 80., 91., 84.])



In [149]:
F_min, result = dual_simplex(A, b, c, M=M)
print(f'F_min = {F_min}')
print(f'Оптимальное значение переменных:\n{result}')
np.sum(result[:len(c)] * c)

Промежуточный F_min = 701000.0
Δ = [-85297.4 -96796.9 -67797.8 -68396.3 -90296.7 -97796.    1000.    1000.
   1000.    1000.    1000.    1000.    1000.    1000. ]
simpex table: 
     0       1     2     3     4     5     6     7     8    9   ...   16   17  \
0  14.0  1000.0  92.0   4.3  13.7  12.9   6.4  13.3   8.9 -1.0  ...  0.0  1.0   
1  15.0  1000.0  95.0   7.3  12.6   9.9   7.4  16.8  15.8  0.0  ...  0.0  0.0   
2  16.0  1000.0  64.0  15.0  14.1   4.0   8.3   7.9  15.6  0.0  ...  0.0  0.0   
3  17.0  1000.0  97.0  11.6  11.3   8.6  15.5  11.4   7.3  0.0  ...  0.0  0.0   
4  18.0  1000.0  98.0  15.3   3.3   9.6  10.0   4.6  11.9  0.0  ...  0.0  0.0   
5  19.0  1000.0  80.0   7.7  12.9   3.7   4.3  13.9  11.2  0.0  ...  0.0  0.0   
6  20.0  1000.0  91.0  14.6  16.4  15.0   9.2  12.9  16.3  0.0  ...  0.0  0.0   
7  21.0  1000.0  84.0   9.5  12.5   4.1   7.3   9.5  10.8  0.0  ... -1.0  0.0   

    18   19   20   21   22   23   24     25  
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  10.34  


np.float64(25.671)